In [ ]:
%reset -f
import importlib
import numpy as np
import matplotlib.pyplot as plt

import sys

import Frog
import Frog.toolbox as toolbox
import Frog.frog_data_analysis as frog_data_analysis

import pickle
import time
from scipy.optimize import curve_fit


In [ ]:
def rotate_4th_order_tensor(base_changement, tensor):
    ''''
    This function transform a 4rd order tensor (3X3X3X3 matrix) from one base to another using a rotational matrix called base_changement.
    If the base_changement is defined to be: vector_{lab} = base_changement . vector_{mol}
    Then: tensor_{lab} = rotate_4th_order_tensor(base_changement.T, tensor_{mol})
    Then: tensor_{mol} = rotate_4th_order_tensor(base_changement, tensor_{lab})
    '''
    tensor_new_base = np.zeros((3, 3, 3, 3))                
    for i in range(0, 3, 1):
        for j in range(0, 3, 1):
            for k in range(0, 3, 1):
                for l in range(0, 3, 1):
                    for p in range(0, 3, 1):
                        for m in range(0, 3, 1):
                            for n in range(0, 3, 1):
                                for o in range(0, 3, 1):
                                    tensor_new_base[i][j][k][l] += base_changement[i][p]*base_changement[j][m]*base_changement[k][n]*base_changement[l][o]*tensor[p][m][n][o]
    return(tensor_new_base)


# Load the gamma

In this example, several static field $E$ have been applied in the laboratory frame while the hyperpolarizability has been computed $\chi[E]$. 
Hence, the second hyperpolarizability $\Gamma$ in the laboratory frame is computed using the first hyperpolarizability (with extra electrostatic field) in the laboratory frame: 

$$\chi[E] = \chi[E=0] + \Gamma . E$$

In [ ]:
dir_ref = [WORK_DIR + 'MD/Water/Bulk/L_90/Analysis/Gamma/Ref'] # the result without extra electrostatic field

L_electric_field = [0.0, 0.0001, 0.0002, 0.0004, 0.0008, 0.0012, 0.0015] # the electrostatic field applied in every direction
dir_electro = WORK_DIR + 'MD/Water/Bulk/L_90/Analysis/Gamma' # Where are the calculation with extra electrostatic field 

N_molecule = 299 # the number of molecule where every QM calculations has been mde
T_steps = 8 # the numùber of time step

N_total = N_molecule*T_steps

L_gamma_lab = np.zeros((N_total, 3, 3, 3, 3)) # the second hyperpolarizability in the laboratory frame
L_gamma_mol = np.zeros((N_total, 3, 3, 3, 3))# the second hyperpolarizability in the molecular frame
L_rot_mat = np.zeros((N_total, 3, 3)) # the rotational matrix of every molecule 

L_xyz = ['x', 'y', 'z']

for TTT in range(0, T_steps, 1):
    print('Time step:', TTT, 'over', T_steps)
    # for static field along x,y and z
    for lll in range(3):
        L_chi_all = np.zeros((N_molecule, 3, 3, 3, len(L_electric_field)))
        axis_electric_field = L_xyz[lll]
        L_directory = [dir_ref] # the result without extra electrostatic field
        for k in range(1, len(L_electric_field), 1):
            L_directory.append(dir_electro + '/E_' + axis_electric_field + '_' + str(L_electric_field[k]))
        # load the data 
        for K in range(0, len(L_electric_field), 1):
            L_moleculetype_result = Frog.toolbox.open_pickle('L_moleculetype_' + str(TTT) + '.p', directory=L_directory[K] + '/Molecule_times')                    
            for n in range(N_molecule):
                for iii in range(3):
                    for jjj in range(3):
                        for kkk in range(3):
                            L_chi_all[n][iii][jjj][kkk][K] = L_moleculetype_result[0].L_molecule[n].chi_0_05686[iii][jjj][kkk]
        # perform linear fit: the slope is the second hyperpolarizability 
        for n in range(N_molecule):
            for iii in range(3):
                for jjj in range(3):
                    for kkk in range(3):
                        coeff = np.polyfit(L_electric_field, L_chi_all[n][iii][jjj][kkk], 1)
                        L_gamma_lab[n+TTT*N_molecule][iii][jjj][kkk][lll] =  coeff[0]


# From laboratory to molecular frame

In [ ]:
for TTT in range(0, T_steps, 1):
    print('Time step:', TTT, 'over', T_steps)
    L_moleculetype_result = Frog.toolbox.open_pickle('L_moleculetype_' + str(TTT) + '.p', directory=dir_ref + '/Molecule_times')                    
    for n in range(N_molecule):
        L_rot_mat[n+TTT*N_molecule] = L_moleculetype_result[0].L_molecule[n].rot_mat
        L_gamma_mol[n+TTT*N_molecule] = rotate_4th_order_tensor(L_rot_mat[n+TTT*N_molecule],  L_gamma_lab[n+TTT*N_molecule]) 

